In [1]:
import reskit as rk
import pandas as pd
import pvlib
import geokit as gk
import xarray as xr
import netCDF4 as nc
from reskit import WorkflowManager

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
placements = pd.DataFrame()
placements['name'] = ['Building1', 'Building2', 'Building3', 'Building4', 'Building5']
placements['lon'] = [ 36.24558, 36.24583, 36.24609, 36.24623, 36.24637]     # Longitude
placements['lat'] = [ 28.16104, 28.16113, 28.16120, 28.16137, 28.16109]    # Latitude
placements['tilt'] = [ 90, 90, 90, 90, 90]   # System tilt in degrees 90 for wall
placements['azimuth'] = [ 250, 250, 250, 250, 250 ]# System azimuth in degrees (orientation) south (SSW) facing
placements['capacity'] = [ 0, 0, 0, 0, 2.429 ] # Total system capacity in kW for each building assuming 2sqm panels
placements['elev'] = [ 0, 0, 0, 0, 0 ]      # Altitute in meters

placements

,name,lon,lat,tilt,azimuth,capacity,elev
0,Building1,36.24558,28.16104,90,250,0.000,0
1,Building2,36.24583,28.16113,90,250,0.000,0
2,Building3,36.24609,28.16120,90,250,0.000,0
3,Building4,36.24623,28.16137,90,250,0.000,0
4,Building5,36.24637,28.16109,90,250,2.429,0


In [3]:
src_sarah=rk.weather.SarahSource(r"/Users/zaramulholland/reskit/reskit/_test/data/SARAH_try"),
src_era5=rk.weather.Era5Source(r"/Users/zaramulholland/reskit/reskit/_test/data/ERA5_try")

/Users/zaramulholland/reskit/reskit/_test/data/SARAH_try/SARAH-DNI.rhodes.y2016.nc
/Users/zaramulholland/reskit/reskit/_test/data/SARAH_try/SARAH-SIS.rhodes.y2016.nc
/Users/zaramulholland/reskit/reskit/_test/data/ERA5_try/reanalysis-era5-single-levels.rhodes.y2016.10m_wind_speed.processed.nc
/Users/zaramulholland/reskit/reskit/_test/data/ERA5_try/reanalysis-era5-single-levels.rhodes.y2016.2m_dewpoint_temperature.nc
/Users/zaramulholland/reskit/reskit/_test/data/ERA5_try/reanalysis-era5-single-levels.rhodes.y2016.2m_temperature.nc
/Users/zaramulholland/reskit/reskit/_test/data/ERA5_try/reanalysis-era5-single-levels.rhodes.y2016.surface_pressure.nc


In [ ]:
sim_result = rk.solar.openfield_pv_sarah_unvalidated(
    placements=placements,
    sarah_path=r"/Users/zaramulholland/reskit/reskit/_test/data/SARAH_try",
    era5_path=r"/Users/zaramulholland/reskit/reskit/_test/data/ERA5_try",
    module='BIPV_BIPV054_T86',
    tracking='fixed'
)



In [ ]:
output_df = sim_result['capacity_factor'].fillna(0).to_dataframe().unstack()


In [ ]:
output_df.index = output_df.index.tz_localize(tz='UTC').tz_convert(tz='Europe/Athens')

In [ ]:
output_df.columns = [('capacity_factor', 'Building1'),
            ('capacity_factor', 'Building2'), 
            ('capacity_factor', 'Building3'),
            ('capacity_factor', 'Building4'),
            ('capacity_factor', 'Building5')]

In [ ]:
output_df.columns

In [ ]:
output_df = output_df.resample('h').sum()

In [ ]:
#output_df.to_csv("/Users/zaramulholland/reskit/PV_SOUTH.csv")

In [ ]:
output_df =sim_result['total_system_generation'].mean("location")

plt.plot(output_df, color='black')

plt.ylabel("System generation (kW)", )
plt.rcParams.update({'font.size': 12})
#positions = (744, 1416, 2160, 2880, 3624, 4344, 5088, 5832, 6552, 7296, 8016, 8760)
positions = (744, 2160, 3648, 5088, 6576, 8100, 9504, 10944, 12432, 13873, 15360, 16806)
labels = ("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec")
plt.xticks(positions, labels)
plt.ylim(0, 0.35)
plt.savefig("SOUTH_Generation", dpi=800)